In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datacompy
import os, sys
import numpy as np

# narzedzia
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    cross_val_predict,
    learning_curve,
    RepeatedStratifiedKFold,
    GridSearchCV
)
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import tree
from sklearn.tree import plot_tree


# modele
from xgboost import XGBClassifier, XGBRFClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

#methods
from imblearn.under_sampling import (
    NearMiss,
    ClusterCentroids
)
from scipy.optimize import differential_evolution
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
# w petli bedzie wykonywala sie np decision tree, ale kazda iteracja bedzie usunieciem danego wiersza i sprawdzeniem jak to wplywa na wynik

### DATA


In [3]:
original_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\original_data.csv")
smote_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\smote_data.csv")
GAN_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\GAN_data.csv")
borderline_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\borderlinesmote_data.csv")

# test data
X_test = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\Test\\X_test.csv")
y_test = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\Test\\y_test.csv")

In [ ]:
#sum synthetic data
sum_syn_data = pd.concat([smote_data, borderline_data], axis=0)
sum_syn_data = pd.concat([sum_syn_data, GAN_data], axis=0)

sum_syn_data = sum_syn_data.reset_index(drop=True)

# sum all data
sum_data = pd.concat([original_data, sum_syn_data], axis=0)
sum_data = sum_data.reset_index(drop=True)
sum_data = sum_data.drop(columns=["Unnamed: 0"])

#X and y 
X_train = sum_data.drop(columns=['target', "source"])
y_train = sum_data['target']

In [34]:
con = original_data['target'].value_counts()
print("Przed", con)
print(original_data.shape)
print(sum_syn_data.shape)

print(sum_data.shape)

con4 = Counter(y)
print("Po", con4)

Przed target
0    163
1     53
Name: count, dtype: int64
(216, 13)
(330, 12)
(546, 12)
Po Counter({1: 383, 0: 163})
